# EMIPredict_AI

**Author:** Mahima Patel

**Objective:** Build a complete pipeline to predict EMI eligibility (classification) and maximum monthly EMI (regression). This notebook includes data cleaning, exploratory data analysis (15+ charts), hypothesis testing (T-test, Chi-square, Pearson correlation, ANOVA), feature engineering, model training (Logistic Regression, RandomForest, XGBoost for classification; Linear Regression, RandomForestRegressor, XGBoostRegressor for regression), and saving artifacts for deployment.

**Notes:**
- Designed to run locally or in Google Colab.
- Uses a sampling option (`SAMPLE_ROWS`) to speed up processing on large datasets.
- Outputs are saved under `artifacts/`:
  - `artifacts/eda_charts/` — PNG charts
  - `artifacts/models/` — saved joblib models and scaler
  - `artifacts/reports/` — EDA report and performance reports


In [ ]:
# Imports & Settings
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import joblib

# plotting defaults
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = (9,5)
plt.rcParams['figure.dpi'] = 120

# Artifact folders
os.makedirs('artifacts/eda_charts', exist_ok=True)
os.makedirs('artifacts/models', exist_ok=True)
os.makedirs('artifacts/reports', exist_ok=True)

# User-editable settings
DATA_PATH = 'emi_prediction_dataset.csv'  # change if your file name differs
SAMPLE_ROWS = 50000  # set to None to use full dataset (careful with large files)
RANDOM_STATE = 42


In [ ]:
# Load dataset (with optional sampling)
print('Loading dataset from:', DATA_PATH)
df = pd.read_csv(DATA_PATH, low_memory=False)
print('Original shape:', df.shape)

if SAMPLE_ROWS and df.shape[0] > SAMPLE_ROWS:
    df = df.sample(n=SAMPLE_ROWS, random_state=RANDOM_STATE).reset_index(drop=True)
    print('Sampled shape:', df.shape)

# Quick preview
display(df.head())
print('\nColumn data types:')
print(df.dtypes)


In [ ]:
# Coerce numeric-like columns and detect categorical columns
numeric_cols = []
for c in df.columns:
    coerced = pd.to_numeric(df[c], errors='coerce')
    if coerced.notnull().mean() > 0.7:
        df[c] = coerced
        numeric_cols.append(c)
cat_cols = [c for c in df.columns if c not in numeric_cols]

print('Detected numeric columns (sample):', numeric_cols[:30])
print('Detected categorical columns (sample):', cat_cols[:30])


In [ ]:
# Missing values & basic cleaning
from sklearn.impute import SimpleImputer

num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')

if numeric_cols:
    df[numeric_cols] = num_imputer.fit_transform(df[numeric_cols])
if cat_cols:
    df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])

# Drop exact duplicates
before = df.shape[0]
df = df.drop_duplicates().reset_index(drop=True)
after = df.shape[0]
print(f'Dropped {before-after} duplicate rows. New shape: {df.shape}')


In [ ]:
# Feature engineering
# Create debt_to_income, total_monthly_expenses, expense_to_income when possible
if set(['monthly_salary','current_emi_amount']).issubset(df.columns):
    df['debt_to_income'] = df['current_emi_amount'] / (df['monthly_salary'] + 1e-6)

if set(['current_emi_amount','other_monthly_expenses']).issubset(df.columns):
    df['total_monthly_expenses'] = df[['current_emi_amount','other_monthly_expenses']].sum(axis=1)

if 'total_monthly_expenses' in df.columns and 'monthly_salary' in df.columns:
    df['expense_to_income'] = df['total_monthly_expenses'] / (df['monthly_salary'] + 1e-6)

# Create credit buckets if credit_score exists
if 'credit_score' in df.columns:
    bins=[300,500,650,700,750,850]
    df['credit_bucket'] = pd.cut(df['credit_score'], bins=bins, include_lowest=True)

print('Feature engineering complete. New columns (if created):', [c for c in ['debt_to_income','total_monthly_expenses','expense_to_income','credit_bucket'] if c in df.columns])


In [ ]:
# Encode categorical columns with LabelEncoder (for modeling)
label_encoders = {}
for c in cat_cols:
    try:
        le = LabelEncoder()
        df[c] = le.fit_transform(df[c].astype(str))
        label_encoders[c] = le
    except Exception as e:
        print('Could not encode', c, e)

# Save encoders
joblib.dump(label_encoders, 'artifacts/models/label_encoders.joblib')
print('Saved label encoders to artifacts/models/label_encoders.joblib')


## Exploratory Data Analysis (EDA)
We will create 15+ charts and save them into `artifacts/eda_charts/`. Each chart will have a short interpretation hint in the notebook for you to copy into the final report.


In [ ]:
# EDA: Generate charts (>=15)
charts = []

def save_fig(fig, name):
    path = os.path.join('artifacts/eda_charts', name)
    fig.tight_layout(); fig.savefig(path); plt.close(fig)
    charts.append(path)

# 1-8: histograms for top numeric features
top_nums = numeric_cols[:8]
for c in top_nums:
    fig = plt.figure()
    plt.hist(df[c].dropna(), bins=30)
    plt.title(f'Distribution: {c}')
    plt.xlabel(c); plt.ylabel('count')
    save_fig(fig, f'chart_hist_{c}.png')

# 9: EMI scenario distribution (bar)
if 'emi_scenario' in df.columns:
    fig = plt.figure()
    vals = df['emi_scenario'].value_counts()
    plt.bar(vals.index.astype(str), vals.values)
    plt.title('EMI scenario distribution')
    save_fig(fig, 'chart_emi_scenario.png')

# 10: monthly_salary histogram
if 'monthly_salary' in df.columns:
    fig = plt.figure(); plt.hist(df['monthly_salary'], bins=40)
    plt.title('Monthly salary distribution'); save_fig(fig,'chart_salary.png')

# 11: requested_amount by emi_scenario (boxplot)
if 'requested_amount' in df.columns and 'emi_scenario' in df.columns:
    fig = plt.figure(figsize=(10,4))
    uniq = df['emi_scenario'].unique()[:10]
    groups = [df[df['emi_scenario']==g]['requested_amount'].values for g in uniq]
    plt.boxplot(groups, labels=[str(g) for g in uniq])
    plt.title('Requested amount by EMI scenario (top 10)')
    save_fig(fig,'chart_req_by_scenario.png')

# 12: salary vs max_monthly_emi scatter
if 'monthly_salary' in df.columns and 'max_monthly_emi' in df.columns:
    fig = plt.figure(); plt.scatter(df['monthly_salary'], df['max_monthly_emi'], alpha=0.3)
    plt.title('Salary vs max_monthly_emi'); save_fig(fig,'chart_salary_vs_emi.png')

# 13: correlation heatmap
fig = plt.figure(figsize=(12,10))
corr = df.select_dtypes(include=[np.number]).corr()
sns.heatmap(corr, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation matrix (numeric features)')
save_fig(fig,'chart_corr.png')

# 14-15: debt_to_income and expense_to_income histograms
for col in ['debt_to_income','expense_to_income']:
    if col in df.columns:
        fig = plt.figure(); plt.hist(df[col].dropna(), bins=30); plt.title(col); save_fig(fig,f'chart_{col}.png')

# 16: credit_bucket counts
if 'credit_bucket' in df.columns:
    fig = plt.figure(); vals = df['credit_bucket'].value_counts().sort_index(); plt.bar([str(i) for i in vals.index], vals.values); plt.title('Credit buckets'); save_fig(fig,'chart_credit_buckets.png')

# 17: gender pie
if 'gender' in df.columns:
    fig = plt.figure(); vals = df['gender'].value_counts(); plt.pie(vals.values, labels=[str(i) for i in vals.index], autopct='%1.1f%%'); plt.title('Gender distribution'); save_fig(fig,'chart_gender.png')

# 18: years_of_employment vs avg max_monthly_emi
if 'years_of_employment' in df.columns and 'max_monthly_emi' in df.columns:
    grp = df.groupby('years_of_employment')['max_monthly_emi'].mean().reset_index()
    fig = plt.figure(); plt.plot(grp['years_of_employment'], grp['max_monthly_emi'], marker='o'); plt.title('Avg max_monthly_emi by years_of_employment'); save_fig(fig,'chart_years_vs_emi.png')

# 19: dependents vs eligibility stacked bar
if 'dependents' in df.columns and 'emi_eligibility' in df.columns:
    cross = pd.crosstab(df['dependents'], df['emi_eligibility'])
    fig = plt.figure(); cross.plot(kind='bar', stacked=True); plt.title('Dependents vs EMI eligibility'); plt.tight_layout(); save_fig(fig,'chart_dependents_vs_eligibility.png')

print('Charts saved:', len(charts))
print('\nSample saved chart paths:')
for p in charts[:10]:
    print('-', p)

# save simple EDA report file
with open('artifacts/reports/EDA_report.md','w') as f:
    f.write('# EDA Report\n\n')
    f.write('Charts generated:\n')
    for p in charts:
        f.write(f'- {p}\n')
    f.write('\n\nInterpretations: add your observations under each chart in this file.')


## Hypothesis Testing
We perform 4 hypothesis tests with clear null and alternative hypotheses:

1. **T-test**: Mean monthly salary differs between eligible and not-eligible applicants.
2. **Chi-square**: Gender and EMI eligibility are dependent.
3. **Pearson correlation**: Credit score correlates with max_monthly_emi.
4. **ANOVA**: Mean debt_to_income differs across employment types.

Each test outputs the test statistic, p-value, and interpretation (reject/fail to reject H0 at α=0.05).


In [ ]:
# Hypothesis Testing
ALPHA = 0.05

# 1: T-test (salary between eligible and not eligible)
if 'monthly_salary' in df.columns and 'emi_eligibility' in df.columns:
    # try to treat emi_eligibility as binary (0/1) or encoded
    unique_vals = sorted(df['emi_eligibility'].unique())
    if len(unique_vals) >= 2:
        # pick first two groups for comparison
        g1 = df[df['emi_eligibility'] == unique_vals[0]]['monthly_salary']
        g2 = df[df['emi_eligibility'] == unique_vals[-1]]['monthly_salary']
        tstat, pval = stats.ttest_ind(g1.dropna(), g2.dropna(), equal_var=False)
        print('T-test: monthly_salary between groups', unique_vals[0], 'and', unique_vals[-1])
        print('t-statistic =', round(tstat,4), ', p-value =', round(pval,6))
        if pval < ALPHA:
            print('Result: Reject H0 -> mean salaries differ (p <', ALPHA,')')
        else:
            print('Result: Fail to reject H0 -> no evidence of difference (p >=', ALPHA,')')

# 2: Chi-square (gender vs eligibility)
if 'gender' in df.columns and 'emi_eligibility' in df.columns:
    ct = pd.crosstab(df['gender'], df['emi_eligibility'])
    chi2, p, dof, ex = stats.chi2_contingency(ct)
    print('\nChi-square test: gender vs emi_eligibility')
    print('chi2 =', round(chi2,4), ', p-value =', round(p,6))
    if p < ALPHA:
        print('Result: Reject H0 -> gender and eligibility are dependent')
    else:
        print('Result: Fail to reject H0 -> no evidence of dependence')

# 3: Pearson correlation (credit_score vs max_monthly_emi)
if 'credit_score' in df.columns and 'max_monthly_emi' in df.columns:
    r, p = stats.pearsonr(df['credit_score'].fillna(0), df['max_monthly_emi'].fillna(0))
    print('\nPearson correlation: credit_score vs max_monthly_emi')
    print('r =', round(r,4), ', p-value =', round(p,6))
    if p < ALPHA:
        print('Result: Reject H0 -> significant correlation exists (p <', ALPHA,')')
    else:
        print('Result: Fail to reject H0 -> no significant correlation')

# 4: ANOVA (debt_to_income across employment_type)
if 'debt_to_income' in df.columns and 'employment_type' in df.columns:
    groups = [g['debt_to_income'].values for n,g in df.groupby('employment_type') if len(g) > 5]
    if len(groups) > 1:
        fstat, p = stats.f_oneway(*groups)
        print('\nANOVA: debt_to_income across employment_type groups')
        print('F =', round(fstat,4), ', p-value =', round(p,6))
        if p < ALPHA:
            print('Result: Reject H0 -> at least one group mean differs')
        else:
            print('Result: Fail to reject H0 -> no evidence of differences')


In [ ]:
## Prepare features for modeling
# Choose explicit targets (if present) and prepare numeric features
class_col = 'emi_eligibility' if 'emi_eligibility' in df.columns else None
reg_col = 'max_monthly_emi' if 'max_monthly_emi' in df.columns else None

# Select numeric features only for a simple baseline pipeline
features = df.select_dtypes(include=[np.number]).columns.tolist()
# remove targets
for t in [class_col, reg_col]:
    if t in features:
        features.remove(t)

X = df[features].fillna(0)
print('Number of numeric features used:', X.shape[1])

# scale
scaler = StandardScaler()
Xs = scaler.fit_transform(X)
joblib.dump(scaler, 'artifacts/models/scaler.joblib')
print('Saved scaler to artifacts/models/scaler.joblib')


In [ ]:
## Classification models (EMI eligibility)
if class_col:
    y = df[class_col].astype(int)
    Xtr, Xte, ytr, yte = train_test_split(Xs, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
    class_models = {
        'LogisticRegression': LogisticRegression(max_iter=1000),
        'RandomForest': RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE),
        'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', n_estimators=100, random_state=RANDOM_STATE)
    }
    class_results = []
    for name, m in class_models.items():
        print('\nTraining', name)
        m.fit(Xtr, ytr)
        preds = m.predict(Xte)
        probs = None
        try:
            probs = m.predict_proba(Xte)
        except:
            probs = None
        acc = accuracy_score(yte, preds)
        prec = precision_score(yte, preds, average='weighted', zero_division=0)
        rec = recall_score(yte, preds, average='weighted', zero_division=0)
        f1s = f1_score(yte, preds, average='weighted', zero_division=0)
        roc = None
        if probs is not None and probs.shape[1] > 1:
            try:
                roc = roc_auc_score(pd.get_dummies(yte), probs, average='weighted', multi_class='ovr')
            except:
                roc = None
        class_results.append({'model':name,'accuracy':acc,'precision':prec,'recall':rec,'f1':f1s,'roc_auc':roc})
        joblib.dump(m, f'artifacts/models/{name}_classification.joblib')
    class_df = pd.DataFrame(class_results)
    display(class_df)
    class_df.to_excel('artifacts/reports/classification_performance.xlsx', index=False)
    print('Saved classification performance to artifacts/reports/classification_performance.xlsx')
else:
    print('No classification target found (emi_eligibility)')


In [ ]:
## Regression models (Max monthly EMI)
if reg_col:
    y = df[reg_col].astype(float)
    Xtr, Xte, ytr, yte = train_test_split(Xs, y, test_size=0.2, random_state=RANDOM_STATE)
    reg_models = {
        'LinearRegression': LinearRegression(),
        'RandomForestRegressor': RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE),
        'XGBoostRegressor': XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=RANDOM_STATE)
    }
    reg_results = []
    for name, m in reg_models.items():
        print('\nTraining', name)
        m.fit(Xtr, ytr)
        preds = m.predict(Xte)
        rmse = np.sqrt(mean_squared_error(yte, preds))
        mae = mean_absolute_error(yte, preds)
        r2 = r2_score(yte, preds)
        mape = (np.mean(np.abs((yte - preds) / (yte + 1e-6)))) * 100
        reg_results.append({'model':name,'rmse':rmse,'mae':mae,'r2':r2,'mape':mape})
        joblib.dump(m, f'artifacts/models/{name}_regression.joblib')
    reg_df = pd.DataFrame(reg_results)
    display(reg_df)
    reg_df.to_excel('artifacts/reports/regression_performance.xlsx', index=False)
    print('Saved regression performance to artifacts/reports/regression_performance.xlsx')
else:
    print('No regression target found (max_monthly_emi)')


In [ ]:
# Final artifacts list
import glob
artifacts = glob.glob('artifacts/**', recursive=True)
print('Artifacts (selected):')
for a in artifacts:
    if a.count('/') <= 2: continue
    print('-', a)

print('\nNotebook run complete. Check artifacts/ for charts, models, and reports.')


## Final notes & next steps
- This notebook provides a full baseline pipeline. For a high-scoring submission, add detailed interpretations under each EDA chart in `artifacts/reports/EDA_report.md`.
- Next improvements (recommended):
  - Hyperparameter tuning with `GridSearchCV` or `Optuna`.
  - Use MLflow to log experiments and compare runs.
  - Build a REST API with FastAPI for model serving.
  - Add SHAP or permutation feature importance for explainability.

**Deliverables produced by this notebook:**
- `artifacts/eda_charts/` (>=15 PNGs)
- `artifacts/models/` (.joblib models + scaler + encoders)
- `artifacts/reports/` (performance Excel files and EDA report)

Good luck — want me to run a sample execution here to generate artifacts (50k sample)?